In [1]:
import pymongo as pm
from pymongo import MongoClient
import configparser

# Connect to MongoDB;

In [2]:
def connect_mongodb(configfile="myconfig.ini", coll_name=None):

# Config Parser;
    config = configparser.ConfigParser()
    config.read(configfile)
# MongoLab OAuth;
    client_host = config.get('mongodb', 'client_host')
    client_port = int(config.get('mongodb', 'client_port'))
    db_name = config.get('mongodb', 'db_name')
    if coll_name is None:
        coll_name = config.get('mongodb', 'coll_name')
    if config.has_option('mongodb', 'user_name'):
        user_name = config.get('mongodb', 'user_name')
    if config.has_option('mongodb', 'passwd'):
        passwd = config.get('mongodb', 'passwd')
# Connect to database;
    try:
        connection = pm.MongoClient(client_host, client_port)
        rlvdb = connection[db_name]  # Database
        if ('user_name' in locals()) and ('passwd' in locals()):
            rlvdb.authenticate(user_name, passwd)
        rlvcl = rlvdb[coll_name]  # Collection
    except Exception:
        sys.exit("Database connection failed!")

    return rlvdb, rlvcl

In [3]:
myalldata, all_data_clusters = connect_mongodb(configfile='myalldata.ini', coll_name='all_data_clusters')

In [4]:
myalldata2, tagged_clusters = connect_mongodb(configfile='myalldata.ini', coll_name='eva_data')

# Tag the clusters and write to DB;

In [5]:
def tag_clusters(input_collection, tagged_collection):

    i = 0

    for cluster in input_collection.find():
    
        i += 1
    
        print("\n\nSome tweets in the cluster " + str(i) + " ; \n")
    
        for tweet in cluster["ctweettuplelist"][:5]:
            print(tweet[2])

        a = len(cluster["ctweettuplelist"])
      
        for tweet in cluster["ctweettuplelist"][a-5:a]:
            print(tweet[2])

        objID = cluster["_id"]
    
        if (tagged_collection.find({"_id" : objID}).count() > 0):

            print("\nWarning: These tweets are already tagged as \"" 
                    + tagged_collection.find_one({"_id" : objID})["class"] + "\".")

            while True:
                check_it = (input("Are they tagged correctly? [Y/n]: ") or "y").lower()
                if check_it not in ["y","yes", "n", "no"]:
                    print("\nError: Argument is invalid. Please enter again [Y/n]:")
                else:
                    break

            if (check_it in ["y","yes"]):

                continue;

            elif (check_it in ["n", "no"]):

                tagged_collection.remove({"_id" : objID})
            
                modified = input("The corret tag of these tweets is... (test, futbol, politics) : ")

                cluster["class"] = modified

                tagged_collection.insert(cluster)
    
           
        else:
            
            tag = input("These tweets are about... (test, futbol, politics) : ")
            
            cluster["class"] = tag
            
            tagged_collection.insert(cluster)
    

    print("All clusters are tagged.")

In [6]:
tag_clusters(all_data_clusters, tagged_clusters)



Some tweets in the cluster 1 ; 

Test
Test
Test
Test
Test
*Test
Test
Test
Test
Test

Are they tagged correctly? [Y/n]: ge

Error: Argument is invalid. Please enter again [Y/n]:
Are they tagged correctly? [Y/n]: Yes


Some tweets in the cluster 2 ; 

test
test
test
test
test
test
test
test
test
test

Are they tagged correctly? [Y/n]: no
The corret tag of these tweets is... (test, futbol, politics) : test


Some tweets in the cluster 3 ; 

Newsletter for test created automatically http://t.co/hl5PdYeeOI http://t.co/gok1cUCWIV
Newsletter for test created automatically http://t.co/hl5PdYeeOI http://t.co/RKBklRkqfx
Newsletter for test created automatically http://t.co/hl5PdYeeOI http://t.co/jdQB5sZC1H
Newsletter for test created automatically http://t.co/hl5PdYeeOI http://t.co/t21OuznJ6o
Newsletter for test created automatically http://t.co/hl5PdYeeOI http://t.co/RTmqFAxG46
Newsletter for test created automatically http://t.co/hl5PdYeeOI http://t.co/XNEE1qHrl9
Newsletter for test created 

KeyboardInterrupt: 

# Remove all data in collection if needed;

In [10]:
tagged_clusters.remove()

{'electionId': ObjectId('55df5b58a487ebf5742f1556'),
 'lastOp': Timestamp(1441960086, 5),
 'n': 4,
 'ok': 1}